# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff3bc761b20>
├── 'a' --> tensor([[ 0.1695,  1.5011, -0.9409],
│                   [-0.2509,  0.1479,  0.5107]])
└── 'x' --> <FastTreeValue 0x7ff3bc7617c0>
    └── 'c' --> tensor([[-1.0038, -0.2612, -0.4258, -0.6569],
                        [-0.9042, -0.3632,  1.0303, -0.1529],
                        [-0.6523,  2.6772, -0.5815,  1.2085]])

In [4]:
t.a

tensor([[ 0.1695,  1.5011, -0.9409],
        [-0.2509,  0.1479,  0.5107]])

In [5]:
%timeit t.a

62.3 ns ± 0.00947 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff3bc761b20>
├── 'a' --> tensor([[-1.4517,  1.2040,  0.6345],
│                   [ 0.1984,  0.7648,  1.0065]])
└── 'x' --> <FastTreeValue 0x7ff3bc7617c0>
    └── 'c' --> tensor([[-1.0038, -0.2612, -0.4258, -0.6569],
                        [-0.9042, -0.3632,  1.0303, -0.1529],
                        [-0.6523,  2.6772, -0.5815,  1.2085]])

In [7]:
%timeit t.a = new_value

62.1 ns ± 0.0664 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.1695,  1.5011, -0.9409],
               [-0.2509,  0.1479,  0.5107]]),
    x: Batch(
           c: tensor([[-1.0038, -0.2612, -0.4258, -0.6569],
                      [-0.9042, -0.3632,  1.0303, -0.1529],
                      [-0.6523,  2.6772, -0.5815,  1.2085]]),
       ),
)

In [10]:
b.a

tensor([[ 0.1695,  1.5011, -0.9409],
        [-0.2509,  0.1479,  0.5107]])

In [11]:
%timeit b.a

54.1 ns ± 0.0209 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4164, -1.4420,  0.1245],
               [ 1.9063,  0.0823,  0.6915]]),
    x: Batch(
           c: tensor([[-1.0038, -0.2612, -0.4258, -0.6569],
                      [-0.9042, -0.3632,  1.0303, -0.1529],
                      [-0.6523,  2.6772, -0.5815,  1.2085]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.115 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

838 ns ± 0.523 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 60 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 275 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 338 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff31703f340>
├── 'a' --> tensor([[[ 0.1695,  1.5011, -0.9409],
│                    [-0.2509,  0.1479,  0.5107]],
│           
│                   [[ 0.1695,  1.5011, -0.9409],
│                    [-0.2509,  0.1479,  0.5107]],
│           
│                   [[ 0.1695,  1.5011, -0.9409],
│                    [-0.2509,  0.1479,  0.5107]],
│           
│                   [[ 0.1695,  1.5011, -0.9409],
│                    [-0.2509,  0.1479,  0.5107]],
│           
│                   [[ 0.1695,  1.5011, -0.9409],
│                    [-0.2509,  0.1479,  0.5107]],
│           
│                   [[ 0.1695,  1.5011, -0.9409],
│                    [-0.2509,  0.1479,  0.5107]],
│           
│                   [[ 0.1695,  1.5011, -0.9409],
│                    [-0.2509,  0.1479,  0.5107]],
│           
│                   [[ 0.1695,  1.5011, -0.9409],
│                    [-0.2509,  0.1479,  0.5107]]])
└── 'x' --> <FastTreeValue 0x7ff31703fa90>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 67.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff3bc761c40>
├── 'a' --> tensor([[ 0.1695,  1.5011, -0.9409],
│                   [-0.2509,  0.1479,  0.5107],
│                   [ 0.1695,  1.5011, -0.9409],
│                   [-0.2509,  0.1479,  0.5107],
│                   [ 0.1695,  1.5011, -0.9409],
│                   [-0.2509,  0.1479,  0.5107],
│                   [ 0.1695,  1.5011, -0.9409],
│                   [-0.2509,  0.1479,  0.5107],
│                   [ 0.1695,  1.5011, -0.9409],
│                   [-0.2509,  0.1479,  0.5107],
│                   [ 0.1695,  1.5011, -0.9409],
│                   [-0.2509,  0.1479,  0.5107],
│                   [ 0.1695,  1.5011, -0.9409],
│                   [-0.2509,  0.1479,  0.5107],
│                   [ 0.1695,  1.5011, -0.9409],
│                   [-0.2509,  0.1479,  0.5107]])
└── 'x' --> <FastTreeValue 0x7ff310453e50>
    └── 'c' --> tensor([[-1.0038, -0.2612, -0.4258, -0.6569],
                        [-0.9042, -0.3632,  1.0303, -0.1529],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 77.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.2 µs ± 248 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.1695,  1.5011, -0.9409],
                [-0.2509,  0.1479,  0.5107]],
       
               [[ 0.1695,  1.5011, -0.9409],
                [-0.2509,  0.1479,  0.5107]],
       
               [[ 0.1695,  1.5011, -0.9409],
                [-0.2509,  0.1479,  0.5107]],
       
               [[ 0.1695,  1.5011, -0.9409],
                [-0.2509,  0.1479,  0.5107]],
       
               [[ 0.1695,  1.5011, -0.9409],
                [-0.2509,  0.1479,  0.5107]],
       
               [[ 0.1695,  1.5011, -0.9409],
                [-0.2509,  0.1479,  0.5107]],
       
               [[ 0.1695,  1.5011, -0.9409],
                [-0.2509,  0.1479,  0.5107]],
       
               [[ 0.1695,  1.5011, -0.9409],
                [-0.2509,  0.1479,  0.5107]]]),
    x: Batch(
           c: tensor([[[-1.0038, -0.2612, -0.4258, -0.6569],
                       [-0.9042, -0.3632,  1.0303, -0.1529],
                       [-0.6523,  2.6772, -0.5815,  1.2085]],
         

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 202 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.1695,  1.5011, -0.9409],
               [-0.2509,  0.1479,  0.5107],
               [ 0.1695,  1.5011, -0.9409],
               [-0.2509,  0.1479,  0.5107],
               [ 0.1695,  1.5011, -0.9409],
               [-0.2509,  0.1479,  0.5107],
               [ 0.1695,  1.5011, -0.9409],
               [-0.2509,  0.1479,  0.5107],
               [ 0.1695,  1.5011, -0.9409],
               [-0.2509,  0.1479,  0.5107],
               [ 0.1695,  1.5011, -0.9409],
               [-0.2509,  0.1479,  0.5107],
               [ 0.1695,  1.5011, -0.9409],
               [-0.2509,  0.1479,  0.5107],
               [ 0.1695,  1.5011, -0.9409],
               [-0.2509,  0.1479,  0.5107]]),
    x: Batch(
           c: tensor([[-1.0038, -0.2612, -0.4258, -0.6569],
                      [-0.9042, -0.3632,  1.0303, -0.1529],
                      [-0.6523,  2.6772, -0.5815,  1.2085],
                      [-1.0038, -0.2612, -0.4258, -0.6569],
                      [-0.9042, -

In [28]:
%timeit Batch.cat(batches)

142 µs ± 242 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

305 µs ± 2.04 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
